This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
#API_KEY = 'Qy_U_9dzrJzs9se65y1b'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key='+API_KEY

r = requests.get(url)
r.status_code

200

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = r.json()
json_data['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1 changing the query string for the data in order to only extract data from calendar year 2017
start = '2017-01-01'
end = '2017-12-31'
payload = dict(start_date=start,end_date=end)
p = requests.get(url,params=payload)
#2 saving relevant portion of this new set of responses as dict
afxx_2017_json=p.json()
afxx_2017_txt=p.text
afxx_2017 = dict(afxx_2017_json['dataset_data'])

In [6]:
#defining lists to hold price data and iterating through the dict to build out each respective list

col_names = afxx_2017['column_names']
dates = []
open_price = []
day_high =[]
day_low = []
day_change = []
close_price = []
trade_volume = []

for day in afxx_2017['data']:
    dates.append(day[0])
    open_price.append(day[1])
    day_high.append(day[2])
    day_low.append(day[3])
    close_price.append(day[4])
    trade_volume.append(day[6])

In [7]:
#3 Removing all None datatypes from the open and close price lists before performing max and min functions
None in open_price
open_price1 = [i for i in open_price if i != None]
open_min = min(open_price1)
open_max = max(open_price1)
print('Highest opening price: ' + str(open_max))
print('Lowest opening price: ' + str(open_min))

Highest opening price: 53.11
Lowest opening price: 34.0


In [8]:
#4 performing subtraction through the day_high and day_low list values to capture the daily delta values
for i in range(len(day_high)):
    day_change.append(float(day_high[i])-float(day_low[i]))
max_daily_change = max(day_change)
print('Max Daily Change in Price: ' + str(max_daily_change))

Max Daily Change in Price: 2.8100000000000023


In [9]:
#5 iterating through the close_price list subtracting the next day's close by today's close up until index n-1
day_to_day_delta = []
for day in range(len(close_price)-1):
    day_to_day_delta.append(close_price[day+1]-close_price[day])
max_day_to_day_change = max(day_to_day_delta)
print('Largest difference in daily close prices: ' + str(max_day_to_day_change))

Largest difference in daily close prices: 2.559999999999995


In [10]:
#6 defining a 'mean' (sum_of_list/len_of_list) and using the function on trade_volume
def mean(my_list):
    return float(sum(my_list)/len(my_list))

average_trading_volume = mean(trade_volume)
print('Average Daily Trading Volume: '+str(average_trading_volume))

Average Daily Trading Volume: 89124.33725490196


In [11]:
#7 defining a 'median' (middle number IF len(list) is odd, avg of middle two numberes IF len(list) is even), and used function on trade_volume
def median(my_list):
    n = len(my_list)
    if n%2==0:
        return (sorted(my_list)[n]+sorted(my_list)[n+1])/2
    elif n%2!=0:
        return sorted(my_list)[n//2]
    else:
        return False

median_trading_volume = median(trade_volume)
print('Median Daily Trading Volume: ' + str(median_trading_volume))

Median Daily Trading Volume: 76286.0
